In [166]:
import torch
from torch import nn
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, DataLoader
import scipy.io as sio

In [167]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [168]:
# Load the data mat data
mat_contents = sio.loadmat('matBisep_26.mat')
Cr = mat_contents['data']
Es = mat_contents['label']



In [169]:
print(Cr.shape)

(600000, 26)


In [170]:
Cr_tensor = torch.tensor(Cr, dtype=torch.float32)
ES_tensor = torch.tensor(Es, dtype=torch.float32)

# # In case we have a gpu
Cr_tensor = Cr_tensor.to(device)
ES_tensor = ES_tensor.to(device)


tensor_dataset = TensorDataset(Cr_tensor, ES_tensor)
dataset = DataLoader(tensor_dataset, batch_size=32, shuffle=True)

In [171]:
# Define the neural network architecture
class NeuralNetwork_with_hidden_layer(nn.Module):
    def __init__(self, n_nodes, n_features):
        super().__init__()
        self.layer1 = nn.Linear(n_features, n_nodes)
        self.layer2 = nn.Linear(n_nodes,4)
        self.relu = nn.ReLU()
        self.soft = nn.Softmax(dim=1)
        self.apply(self._init_weights)


    def forward(self, x):
      # if input x provided, describe how to produce output tensor
        x = self.relu(self.layer1(x))
        # x = self.soft(self.layer2(x))
        x = self.layer2(x)
        return x

    def _init_weights(self, l):
      # Initialize weights uniformly
      if isinstance(l, nn.Linear):
        nn.init.kaiming_uniform_(l.weight)
        #nn.init.uniform_(l.weight)
        l.bias.data.fill_(0.01)

In [172]:
ES_tensor.view(-1)

tensor([1., 0., 1.,  ..., 0., 0., 3.])

In [173]:
def train_model(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Train the model
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction error
        y = y.view(-1).long()
        print
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_model(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            # Reshape y to match pred's shape
            y = y.view(-1).long() # Assuming y has shape (batch_size, 1)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [174]:
depth = [1,2,4,8,16,32,64,128]
for i in depth:
    model_simple = NeuralNetwork_with_hidden_layer(i,26).to(device)
    # loss_fn = CustomLoss()
    loss_fn =  nn.CrossEntropyLoss()
    # optimizer = torch.optim.SGD(model_simple.parameters(), lr=0.001, momentum=0.5)
    optimizer = torch.optim.RMSprop(model_simple.parameters(), lr=0.001)
    epochs = 50
    for t in range(epochs):
        # print(f"Epoch {t+1}\n-------------------------------")
        train_model(dataset, model_simple, loss_fn, optimizer)
    test_model(dataset, model_simple, loss_fn)


Epoch 1
-------------------------------
Test Error: 
 Accuracy: 88.0%, Avg loss: 0.319440 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 89.1%, Avg loss: 0.278312 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.260329 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 89.9%, Avg loss: 0.253009 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 90.1%, Avg loss: 0.244441 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 90.2%, Avg loss: 0.240781 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 90.6%, Avg loss: 0.231932 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 90.9%, Avg loss: 0.222012 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 90.8%, Avg loss: 0.222568 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 91.2%, Avg loss: 0.214909 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 91.2%, Avg los

In [175]:
test_model(dataset, model_simple, loss_fn)

Test Error: 
 Accuracy: 92.0%, Avg loss: 0.192844 

